# Training a Simple LUTNN

In [ ]:
import torch
import torch.optim as optim
import os
import sys
from pathlib import Path
from lutnn.lutlayer import LUTLayer, Aggregation

In [ ]:
from utils.mnist import load_mnist_dataset
train_loader, test_loader, input_dim_dataset, num_classes = load_mnist_dataset(batch_size=128, mnist20=True)

In [ ]:
PROJECT_ROOT = Path.cwd().parent  # go up one level

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from hdl.convert2vhdl import get_model_params, gen_vhdl_code
from hdl.convert2sv import gen_sv_code

In [4]:
# Define a simple LUTNN
class SimpleLUTNN(torch.nn.Module):
    def __init__(self):
        super(SimpleLUTNN, self).__init__()
        self.layer1 = LUTLayer(input_dim=input_dim_dataset, lut_size=6, n_luts=2048)
        self.layer2 = LUTLayer(input_dim=2048, lut_size=6, n_luts=4000)
        self.layer3 = Aggregation(num_classes=num_classes, tau = 10.)

    def forward(self, x):
        x = x.view(-1, 20*20)  # Flatten the input
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = SimpleLUTNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

cpu


In [6]:
# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/4], Step [0/468], Loss: 2.3232
Epoch [1/4], Step [100/468], Loss: 0.4826
Epoch [1/4], Step [200/468], Loss: 0.3716
Epoch [1/4], Step [300/468], Loss: 0.2235
Epoch [1/4], Step [400/468], Loss: 0.2060
Epoch [2/4], Step [0/468], Loss: 0.1224
Epoch [2/4], Step [100/468], Loss: 0.1990
Epoch [2/4], Step [200/468], Loss: 0.1396
Epoch [2/4], Step [300/468], Loss: 0.1250
Epoch [2/4], Step [400/468], Loss: 0.0953
Epoch [3/4], Step [0/468], Loss: 0.0974
Epoch [3/4], Step [100/468], Loss: 0.1864
Epoch [3/4], Step [200/468], Loss: 0.0645
Epoch [3/4], Step [300/468], Loss: 0.0680
Epoch [3/4], Step [400/468], Loss: 0.0968
Epoch [4/4], Step [0/468], Loss: 0.0320
Epoch [4/4], Step [100/468], Loss: 0.0971
Epoch [4/4], Step [200/468], Loss: 0.0590
Epoch [4/4], Step [300/468], Loss: 0.0660
Epoch [4/4], Step [400/468], Loss: 0.0896
Test Accuracy: 97.06%


In [7]:
# Wrap the layers so the HDL export functions can find them
model.model = torch.nn.Sequential(
    torch.nn.Flatten(),
    model.layer1,
    model.layer2,
    model.layer3
)

number_of_layers, num_neurons, lut_sizes, number_of_inputs, number_of_classes = get_model_params(model)
model_name = 'simple_lutnn'

In [8]:
# VHDL generation
gen_vhdl_code(model, model_name, number_of_layers, number_of_classes, number_of_inputs, num_neurons, lut_sizes)
print(f'VHDL code generated in data/VHDL/{model_name}/')

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): LUTLayer(num_luts=2048, lut_size=6)
  (2): LUTLayer(num_luts=4000, lut_size=6)
  (3): Aggregation(num_classes=10, tau=10.0)
)
tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 1, 0,  ..., 1, 0, 0],
        [1, 0, 0,  ..., 1, 1, 0],
        ...,
        [0, 1, 0,  ..., 0, 1, 1],
        [1, 0, 0,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0]])
tensor([[1, 1, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 1],
        ...,
        [1, 1, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
VHDL code generated in data/VHDL/simple_lutnn/


In [9]:
# SV generation
gen_sv_code(model, model_name, number_of_layers, number_of_classes, number_of_inputs, num_neurons, lut_sizes)
print(f'SystemVerilog code generated in data/sv/{model_name}/')

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): LUTLayer(num_luts=2048, lut_size=6)
  (2): LUTLayer(num_luts=4000, lut_size=6)
  (3): Aggregation(num_classes=10, tau=10.0)
)
tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 1, 0,  ..., 1, 0, 0],
        [1, 0, 0,  ..., 1, 1, 0],
        ...,
        [0, 1, 0,  ..., 0, 1, 1],
        [1, 0, 0,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0]])
tensor([[1, 1, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 1],
        ...,
        [1, 1, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
SystemVerilog code generated in data/sv/simple_lutnn/
